In [1]:
## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hi'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
# Setting a temp directory for tests
TMP=/tmp/test-unix

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..

alias | wc -l

0


In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 3245)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-3245

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-unix/test-3245


In [6]:
# Count aliases proper
alias | wc -l
# Count functions
typeset -f | egrep '^\w+' | wc -l

1
0


In [7]:
# ps-all(pattern): show processes from all users matching PATTERN (or . in which case piped to less)
# TODO: have option to restrict to current user
function ps-all () { 
    local pattern="$1";
    local pager=cat;
    if [ "$pattern" = "" ]; then 
        pattern="."; 
        pager=$PAGER
    fi;
    ps_mine.sh --all | $EGREP -i "((^USER)|($pattern))" | $pager;
    }
alias ps-script='ps-all "\\bscript\\b" | $GREP -v "(gnome-session)"'
function ps-sort-once { ps_sort.perl -num_times=1 -by=time "$@" -; }
alias ps-sort-time='ps-sort-once -by=time'
alias ps-time=ps-sort-time
alias ps-sort-mem='ps-sort-once -by=mem '
alias ps-mem=ps-sort-mem


In [8]:
# SORTS ALL PROCESS BY TIME
ps-sort-time

Error: bad sort field (time): using cpu
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
aveey       1279 18.0 12.4 4132464 459444 ?      Sl   20:06  14:33 /snap/firef
aveey       8400 14.6  1.3 672896 51428 ?        Ssl  21:27   0:00 /usr/bin/py
aveey       4661 13.0  6.4 2697144 237140 ?      Sl   20:40   6:09 /snap/firef
aveey       2593  4.9 20.4 3537544 755560 ?      Sl   20:07   3:57 /snap/firef
root         890  4.6  2.7 681536 100956 tty1    Ssl+ 20:05   3:49 /usr/lib/xo
aveey       6783  4.6  8.8 25968212 324836 ?     Sl   20:42   2:04 /opt/Upwork
aveey       8410  1.0  0.1  11412  5508 pts/6    Ss   21:27   0:00 /usr/bin/ba
aveey       6719  0.7  4.4 4831484 165444 ?      Sl   20:42   0:20 /opt/Upwork
aveey       4718  0.6  2.0 243932 75760 pts/0    Sl+  20:41   0:17 /usr/bin/py
aveey       7968  0.6  4.1 2509728 153340 ?      Sl   21:15   0:04 /snap/firef
aveey       2300  0.5  3.8 27820480 141776 ?     Sl   20:07   0:25 /snap/firef
aveey       1070

In [9]:
# ACTS THE SAME AS ps-sort-time
ps-time

Error: bad sort field (time): using cpu
USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
aveey       1279 18.0 12.4 4132464 457624 ?      Sl   20:06  14:34 /snap/firef
aveey       4661 13.1  6.4 2698168 237800 ?      Sl   20:40   6:09 /snap/firef
aveey       8400 12.5  1.3 672896 51428 ?        Ssl  21:27   0:00 /usr/bin/py
aveey       2593  4.9 20.5 3537544 755776 ?      Sl   20:07   3:57 /snap/firef
root         890  4.6  2.7 681536 100956 tty1    Ssl+ 20:05   3:49 /usr/lib/xo
aveey       6783  4.6  8.8 25968212 324836 ?     Sl   20:42   2:04 /opt/Upwork
aveey       8410  0.8  0.1  11412  5508 pts/6    Ss   21:27   0:00 /usr/bin/ba
aveey       6719  0.7  4.4 4831484 165444 ?      Sl   20:42   0:20 /opt/Upwork
aveey       4718  0.6  2.0 243932 75760 pts/0    Sl+  20:41   0:17 /usr/bin/py
aveey       7968  0.6  4.1 2509728 153340 ?      Sl   21:15   0:04 /snap/firef
aveey       2300  0.5  3.8 27820480 141776 ?     Sl   20:07   0:25 /snap/firef
aveey       1070

In [10]:
# SORTS ALL PROCESS BY MEMORY
ps-sort-mem

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
aveey       2593  4.9 20.5 3537544 755804 ?      Rl   20:07   3:57 /snap/firef
aveey       1279 18.0 12.4 4132464 458796 ?      Sl   20:06  14:34 /snap/firef
aveey       6783  4.6  8.8 25968212 324836 ?     Sl   20:42   2:04 /opt/Upwork
aveey       4661 13.1  6.4 2699192 239568 ?      Sl   20:40   6:09 /snap/firef
aveey       6719  0.7  4.4 4831484 165444 ?      Sl   20:42   0:21 /opt/Upwork
aveey       7968  0.6  4.1 2509728 153340 ?      Sl   21:15   0:04 /snap/firef
aveey       2300  0.5  3.8 27820480 141776 ?     Sl   20:07   0:25 /snap/firef
aveey       1889  0.0  2.8 2448140 106052 ?      Sl   20:07   0:02 /snap/firef
root         890  4.6  2.7 681536 100956 tty1    Ssl+ 20:05   3:49 /usr/lib/xo
aveey       7301  0.1  2.7 1000220 101168 ?      Sl   20:52   0:03 /usr/bin/fe
aveey       1070  0.1  2.4 1206000 91340 ?       Sl   20:05   0:06 /usr/bin/lx
aveey       1066  0.0  2.3 861292 85396 ?        Sl   20

In [11]:
# ps-mem IS AN ALTERNATIVE OF ps-sort-mem
ps-mem

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
aveey       2593  4.9 20.5 3537544 756020 ?      Sl   20:07   3:57 /snap/firef
aveey       1279 18.0 12.4 4132464 459512 ?      Sl   20:06  14:34 /snap/firef
aveey       6783  4.6  8.3 25950552 307252 ?     Sl   20:42   2:04 /opt/Upwork
aveey       4661 13.1  6.5 2699192 239908 ?      Sl   20:40   6:10 /snap/firef
aveey       6719  0.7  4.4 4831484 165444 ?      Sl   20:42   0:21 /opt/Upwork
aveey       7968  0.6  4.1 2509728 153340 ?      Sl   21:15   0:04 /snap/firef
aveey       2300  0.5  3.8 27820480 141776 ?     Sl   20:07   0:25 /snap/firef
aveey       1889  0.0  2.8 2448140 106052 ?      Sl   20:07   0:02 /snap/firef
root         890  4.6  2.7 681536 100956 tty1    Ssl+ 20:05   3:49 /usr/lib/xo
aveey       7301  0.1  2.7 1000220 101168 ?      Sl   20:52   0:03 /usr/bin/fe
aveey       1070  0.1  2.4 1206000 91340 ?       Sl   20:05   0:06 /usr/bin/lx
aveey       1066  0.0  2.3 861292 85396 ?        Sl   20

In [12]:
# ERROR GENERATED FOR ps-script
# $ ps-script
# | bash: -v: command not found
# | bash: -i: command not found
# | /usr/bin/env: ‘csh’: No such file or directory


bash: -v: command not found
bash: -i: command not found
/usr/bin/env: ‘csh’: No such file or directory


: 127

In [14]:
# ERROR GENERATED FOR ps-all
# $ ps-all
# | /usr/bin/env: ‘csh’: No such file or directory
# | bash: -i: command not found


/usr/bin/env: ‘csh’: No such file or directory
bash: -i: command not found


In [15]:
function get-process-parent() { local pid="$1"; if [ "$pid" = "" ]; then pid=$$; fi; ps al | perl -Ssw extract_matches.perl "^\d+\s+\d+\s+$pid\s+(\d+)"; }

In [18]:
get-process-parent | egrep "(PID|$$)"

8400


: 1